In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('../Sentiment & Engagement Datasets/ready_data_score.csv')

In [3]:
df.head(2)

,id,text,title,author,num_comments,post_id,upvote_ratio,score,url,subreddit,link_flair_text,link_flair_template_id,created_datetime,day_of_week,hour_of_day,month,year,sentiment_score
0,41034.0,Hi context year old guy Amsterdam currently em...,Lazy job or Hard job?,Weak_Assumption_6889,8,1bfpxll,0.33,0,https://www.reddit.com/r/careeradvice/comments...,careeradvice,Unknown,Unknown,2024-03-15 22:07:22,Friday,22.0,March,2024,0.7579
1,43519.0,Looking new role havenut much traction Recentl...,Roast my Resume Pls,Neither_Trash,1,1bh8md2,0.99,1,https://i.redd.it/n918fjprlyoc1.jpeg,resumes,Review my resume • I'm in North America,c292b8e0-28b9-11ec-874c-325b17e851a3,2024-03-17 21:05:40,Sunday,21.0,March,2024,0.6369


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7862 entries, 0 to 7861
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      7107 non-null   float64
 1   text                    7862 non-null   object 
 2   title                   7862 non-null   object 
 3   author                  7760 non-null   object 
 4   num_comments            7862 non-null   int64  
 5   post_id                 7862 non-null   object 
 6   upvote_ratio            7862 non-null   float64
 7   score                   7862 non-null   int64  
 8   url                     7862 non-null   object 
 9   subreddit               7862 non-null   object 
 10  link_flair_text         7508 non-null   object 
 11  link_flair_template_id  7508 non-null   object 
 12  created_datetime        7862 non-null   object 
 13  day_of_week             7107 non-null   object 
 14  hour_of_day             7107 non-null   